# EEG - Flow

## 6. Epochs Evokeds


Last edit: 17.04.2023 01:57
@anguyen

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
from autoreject import get_rejection_threshold
from matplotlib import pyplot as plt # viz
from scipy.stats import norm
import math

from mne import (
    Epochs, find_events, pick_types, rename_channels, write_events, read_epochs,
    read_evokeds, read_events
)

from mne.epochs import make_metadata
from mne.io import read_raw_fif

from mne.viz import set_browser_backend

from eeg_flow.config import load_config
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files

%matplotlib qt
set_browser_backend('qt')

_, derivatives_folder_root, experimenter = load_config()

Using qt as 2D backend.


The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [2]:
participant = 19  # int
group       = 6  # int
task        = "oddball"  # str
run         = 2  # int

########################################################################################

derivatives_folder = get_folder(derivatives_folder_root, participant, group)
fname_stem = get_fname(participant, group, task, run)

# create locks
derivatives = [
    derivatives_folder / fname_stem / (fname_stem + "_step6_a-metadata.csv"),
    derivatives_folder / fname_stem / (fname_stem + "_step6_b-behav.txt"),
    derivatives_folder / fname_stem / (fname_stem + "_step6_c-events-eve.fif"),
    derivatives_folder / fname_stem / (fname_stem + "_step6_d1-raw-epo.fif"),
    derivatives_folder / fname_stem / (fname_stem + "_step6_d2-cleaned-epo.fif"),
    derivatives_folder / fname_stem / (fname_stem + "_step6_e1-standard_evoked-ave.fif"),
    derivatives_folder / fname_stem / (fname_stem + "_step6_e2-novel_evoked-ave.fif"),
    derivatives_folder / fname_stem / (fname_stem + "_step6_e3-target_evoked-ave.fif"),
]

locks = lock_files(*derivatives)

# load previous steps
## load raw_fit recording
raw = read_raw_fif(derivatives_folder / fname_stem / (fname_stem + "_step5_preprocessed-raw.fif"), preload=True)
# ## load following annots
# # info = read_info(derivatives_folder / fname_stem / (fname_stem + "_step2_info.fif"))
# # annot = read_annotations(derivatives_folder / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif"))

# # merge info and annots into current raw
# raw.info["bads"] = info["bads"]
# raw.set_annotations(annot)

Opening raw data file L:\EEG_Flow_data\derivatives\sub-P19-G6\sub-P19-G6_task-oddball_run-2\sub-P19-G6_task-oddball_run-2_step5_preprocessed-raw.fif...
    Range : 2005 ... 380284 =      1.958 ...   371.371 secs
Ready.
Reading 0 ... 378279  =      0.000 ...   369.413 secs...


## 5.1 Behavioral

In [3]:
events = find_events(raw, stim_channel='TRIGGER')
events

events_id = dict(standard=1, target=2, novel=3, response=64)
row_events = ['standard','target',"novel"]

# metadata for each epoch shall include events from the range: [0.0, 1.5] s,
# i.e. starting with stimulus onset and expanding beyond the end of the epoch
metadata_tmin, metadata_tmax = 0.0, 0.999

# auto-create metadata
# this also returns a new events array and an event_id dictionary. we'll see
# later why this is important
metadata, events, event_id = make_metadata(
    events=events, event_id=events_id,
    tmin=metadata_tmin, tmax=metadata_tmax, sfreq=raw.info['sfreq'], row_events=row_events)

# let's look at what we got!
#metadata.to_csv("metadata.csv")
metadata

conditions = [metadata['event_name'].eq('target') & pd.notna(metadata['response']),
                    metadata['event_name'].eq('target') & pd.isna(metadata['response']),

                    metadata['event_name'].eq('standard') & pd.notna(metadata['response']),    
                    metadata['event_name'].eq('standard') & pd.isna(metadata['response']),

                    metadata['event_name'].eq('novel') & pd.notna(metadata['response']),
                    metadata['event_name'].eq('novel') & pd.isna(metadata['response'])]
choices = ["Hits","Misses","FalseAlarms","CorrectRejections","FalseAlarms","CorrectRejections"]
        


metadata['response_type'] = np.select(conditions, choices, default=0)
metadata['response_type'].value_counts()

hits = metadata[metadata['response_type']=="Hits"]
response_mean = round(hits["response"].mean(),5)
responses_std = round(hits["response"].std(),5)

num_Hits = len(metadata[metadata['response_type'] == "Hits"])
num_CorrectRejections = len(metadata[metadata['response_type'] == "CorrectRejections"])
num_Misses = len(metadata[metadata['response_type'] == "Misses"])
num_FalseAlarms = len(metadata[metadata['response_type'] == "FalseAlarms"])

395 events found
Event IDs: [ 1  2  3 64]


In [4]:
# visualize response times of TP
ax_rt = hits['response'].plot.hist(bins=100, title='Response Times of TPs\nmean:'+str(response_mean)+" ("+str(responses_std)+")")

fname_rt_plot = derivatives_folder / fname_stem / "plots" / (fname_stem + "_step6_RT.svg")
ax_rt.figure.suptitle(fname_stem, fontsize=16, y=1)
ax_rt.figure.savefig(fname_rt_plot)
ax_rt

<Axes: title={'center': 'Response Times of TPs\nmean:0.38711 (0.05456)'}, ylabel='Frequency'>

In [5]:
metadata.response_correct=False
metadata.loc[(metadata['response_type'] == "CorrectRejections"), 'response_correct'] = True
metadata.loc[(metadata['response_type'] == "Hits"), 'response_correct'] = True

metadata.loc[(metadata['response_type'] == "FalseAlarms"), 'response_correct'] = False
metadata.loc[(metadata['response_type'] == "Misses"), 'response_correct'] = False

correct_response_count = metadata['response_correct'].sum()

print(f'Correct responses: {correct_response_count}\n'
      f'Incorrect responses: {len(metadata) - correct_response_count}')

fname_metadata = derivatives_folder / fname_stem / (fname_stem + "_step6_a-metadata.csv")
metadata.to_csv(fname_metadata)

Correct responses: 359
Incorrect responses: 1


In [6]:
Z = norm.ppf

def SDT(hits, misses, fas, crs):
    """ returns a dict with d-prime measures given hits, misses, false alarms, and correct rejections"""
    # Floors an ceilings are replaced by half hits and half FA's
    half_hit = 0.5 / (hits + misses)
    half_fa = 0.5 / (fas + crs)
 
    # Calculate hit_rate and avoid d' infinity
    hit_rate = hits / (hits + misses)
    if hit_rate == 1: 
        hit_rate = 1 - half_hit
    if hit_rate == 0: 
        hit_rate = half_hit
 
    # Calculate false alarm rate and avoid d' infinity
    fa_rate = fas / (fas + crs)
    if fa_rate == 1: 
        fa_rate = 1 - half_fa
    if fa_rate == 0: 
        fa_rate = half_fa
 
    # Return d', beta, c and Ad'
    out = {}
    out['d'] = Z(hit_rate) - Z(fa_rate)
    out['beta'] = math.exp((Z(fa_rate)**2 - Z(hit_rate)**2) / 2)
    out['c'] = -(Z(hit_rate) + Z(fa_rate)) / 2
    out['Ad'] = norm.cdf(out['d'] / math.sqrt(2))
    
    return(out)
print(num_Hits, num_Misses, num_FalseAlarms, num_CorrectRejections)
SDT(num_Hits, num_Misses, num_FalseAlarms, num_CorrectRejections)

35 1 0 324


{'d': 4.873502955557731,
 'beta': 12.745340222708064,
 'c': 0.5222456527233088,
 'Ad': 0.9997156163789449}

In [7]:
metadata.groupby(by="event_name").count()

,standard,target,novel,response,response_type,response_correct
event_name,,,,,,
novel,0,0,36,0,36,36
standard,288,0,0,0,288,288
target,0,36,0,35,36,36


In [8]:
#write behav file

fname_behav = derivatives_folder / fname_stem / (fname_stem + "_step6_b-behav.txt")

file_behav = open(fname_behav,"w")
  
file_behav.write("Hits, Misses, Correct Rejections, False Alarms\n")
file_behav.write(str(num_Hits) + "\t" + str(num_Misses) + "\t " + str(num_CorrectRejections) + "\t" + str(num_FalseAlarms))

file_behav.write("\n\nStandard, Novel, Target\n")
file_behav.write(str(metadata.groupby(by="event_name").count()["response_correct"]["standard"]) + "\t" +
                 str(metadata.groupby(by="event_name").count()["response_correct"]["novel"]) + "\t" +
                 str(metadata.groupby(by="event_name").count()["response_correct"]["target"]))

file_behav.write("\n\nResponse_mean, Response_std\n")
file_behav.write(str(response_mean) + "\t" + str(responses_std))

file_behav.write("\n\nd'\n")
file_behav.write(str(SDT(num_Hits, num_Misses, num_FalseAlarms, num_CorrectRejections)['d']))

file_behav.close() #to change file access modes


In [9]:
from scipy.stats import norm
import math
Z = norm.ppf

def SDT(hits, misses, fas, crs):
    """ returns a dict with d-prime measures given hits, misses, false alarms, and correct rejections"""
    # Floors an ceilings are replaced by half hits and half FA's
    half_hit = 0.5 / (hits + misses)
    half_fa = 0.5 / (fas + crs)
 
    # Calculate hit_rate and avoid d' infinity
    hit_rate = hits / (hits + misses)
    
    # Calculate false alarm rate and avoid d' infinity
    fa_rate = fas / (fas + crs)

    # Return d', beta, c and Ad'
    out = {}
    out['d'] = Z(hit_rate) - Z(fa_rate)
    out['beta'] = math.exp((Z(fa_rate)**2 - Z(hit_rate)**2) / 2)
    out['c'] = -(Z(hit_rate) + Z(fa_rate)) / 2
    out['Ad'] = norm.cdf(out['d'] / math.sqrt(2))
    
    return(out)
SDT(num_Hits, num_Misses, num_FalseAlarms, num_CorrectRejections)

{'d': inf, 'beta': inf, 'c': inf, 'Ad': 1.0}

## 6.2 Epochs


In [12]:
event_id

{'standard': 1, 'target': 2, 'novel': 3}

In [ ]:
epochs_tmin, epochs_tmax = -0.2, 0.8

epochs = Epochs(raw=raw, tmin=epochs_tmin, tmax=epochs_tmax,
                    events=events, event_id=event_id, metadata=metadata,
                    reject=None, preload=True, baseline=(None, 0), picks="eeg")

In [ ]:
fname_event = derivatives_folder / fname_stem / (fname_stem + "_step6_c-events-eve.fif")
write_events(fname_event, events)

In [ ]:
#epochs with no change
fname_raw_epochs = derivatives_folder / fname_stem / (fname_stem + "_step6_d1-raw-epo.fif")
epochs.save(fname_raw_epochs)

In [ ]:
%%time
reject = get_rejection_threshold(epochs, decim=1, ch_types ="eeg", random_state=888)
print('Peak-to-peak rejection threshold computed: %s', reject)


In [ ]:
# reject = dict(# unit: T / m (gradiometers)
#                   # unit: T (magnetometers)
#               eeg=100e-6,      # unit: V (EEG channels)
#                  # unit: V (EOG channels)
#               )


epochs.drop_bad(reject=reject)
fig = epochs.plot_drop_log(subject=fname_stem)

In [ ]:
fname_drop_log = derivatives_folder / fname_stem / "plots" / (fname_stem + "_step6_epochs-rejected.svg")
fig.savefig(fname_drop_log)

In [ ]:
fname_cleaned_epochs = derivatives_folder / fname_stem / (fname_stem + "_step6_d2-cleaned-epo.fif")
epochs.save(fname_cleaned_epochs)

In [ ]:
epochs

In [ ]:
epochs.metadata.groupby(by=["event_name","response_correct",]).count()

In [ ]:
#this keeps correct responses only (hits and correct rejection)
epochs["response_correct"]


In [ ]:
all_evokeds = dict((cond, epochs["response_correct"][cond].average()) for cond in event_id)
all_evokeds

## 6.3 Save the rest of the derivatives

The ICA decomposition can be saved.

In [ ]:
fname_ev_standard = derivatives_folder / fname_stem / (fname_stem + "_step6_e1-standard_evoked-ave.fif")
fname_ev_novel    = derivatives_folder / fname_stem / (fname_stem + "_step6_e2-novel_evoked-ave.fif")
fname_ev_target   = derivatives_folder / fname_stem / (fname_stem + "_step6_e3-target_evoked-ave.fif")

all_evokeds["standard"].save(fname_ev_standard)
all_evokeds["novel"].save(fname_ev_novel)
all_evokeds["target"].save(fname_ev_target)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway